# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#Generate initial data
cities_df = pd.read_csv('../output_data/cities.csv', index_col = False).dropna()
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Conway,35.0887,-92.4421,55.38,81,90,5.99,US,1635557217
1,São João da Barra,-21.6403,-41.0511,70.56,87,100,16.26,BR,1635557407
2,Khatanga,71.9667,102.5000,5.86,85,100,8.03,RU,1635557209
3,Mataura,-46.1927,168.8643,74.50,38,57,5.01,NZ,1635557409
4,Rikitea,-23.1203,-134.9692,75.54,81,100,15.68,PF,1635557409


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure gmap
gmaps.configure(api_key = g_key)

In [5]:
locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"]


In [6]:
#Pulling in Gmaps/generate heat map

fig = gmaps.figure(center=(22.0, -1.0), zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights = weight, 
                                 dissipating = False, max_intensity = np.max(weight),
                                 point_radius = 4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#Generate initial dataframe
cities_new_df = cities_df.dropna()
cities_new_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Conway,35.0887,-92.4421,55.38,81,90,5.99,US,1635557217
1,São João da Barra,-21.6403,-41.0511,70.56,87,100,16.26,BR,1635557407
2,Khatanga,71.9667,102.5000,5.86,85,100,8.03,RU,1635557209
3,Mataura,-46.1927,168.8643,74.50,38,57,5.01,NZ,1635557409
4,Rikitea,-23.1203,-134.9692,75.54,81,100,15.68,PF,1635557409


In [9]:
filtered_cities = cities_new_df.loc[(cities_new_df["Max Temp"] <= 80) & 
                                    (cities_new_df["Max Temp"] >= 70) & 
                                    (cities_new_df["Wind Speed"] < 10) &  
                                    (cities_new_df["Cloudiness"] == 0)]
                                
filtered_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,Coalinga,36.1397,-120.3602,79.34,64,0,2.93,US,1635557418
34,San Patricio,28.0170,-97.5169,72.84,27,0,7.76,US,1635557309
49,Salalah,17.0151,54.0924,77.09,61,0,5.75,OM,1635557434
59,Puerto Peñasco,31.3167,-113.5333,74.91,73,0,6.11,MX,1635557438
60,Santa Cruz de Tenerife,28.4682,-16.2546,71.69,79,0,1.01,ES,1635557439
69,Carpinteria,34.3989,-119.5185,79.36,59,0,2.75,US,1635557445
217,Iranshahr,27.2025,60.6848,71.85,31,0,3.27,IR,1635557501
250,Muscat,23.6139,58.5922,73.42,53,0,2.57,OM,1635557508
355,Porto Velho,-8.7619,-63.9039,75.25,94,0,1.14,BR,1635557558
359,Warah,27.4481,67.7978,73.71,23,0,0.96,PK,1635557560


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = pd.DataFrame(filtered_cities)
hotel_df["Hotel Name"] = ""

In [13]:
#Using API/JSON data to populate hotel information

target_search = "hotels"

target_radius = 5000
target_type = "Business"

params = {
    "keyword": target_search,
    "radius":target_radius,
    "type":target_type,
    "key" : g_key
}
    
for index, row in hotel_df.iterrows():

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    
    hotel_info = requests.get(base_url, params=params)
    hotel_info = hotel_info.json()
    
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_info["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = hotel_info["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Rating"] = hotel_info["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_info

{'html_attributions': [],
 'next_page_token': 'Aap_uECyWa732jyKIjLDTvVHqeG-HqnErw9bfqKAPKAsIpA25QwCeCYxUYhW-fDhEAHEg99acGhnG5W7DduOjFmspUbFdBKucmDM-MrlLVxTWg1L291Xc52B18JLb_hA3Ol4mjXw7JOtwpfzN5IHqIDQ4_deNmWELyecrwELbjfPV7QvVP3I3eUESZiZYtUpIEpdeHGdIgjHiquTpmUhLMCO-JMCK2ae2mPOLWwxdvWN2sq8AEDUMI5q4l_OVixUznD9o1f4pIkzZSNjhWCBopPazNkwJhL5eQfs3O8_ER9UhYVwTuYXezeYVFZ1ITh-8TexAm9_GF6n3IFOIrZBeYg_ipHk5_ACf_HPFuKbmUpuEurjnHOQhNwOeqkSKWevmuyEVk_IwNFnFh1ig10BrYPYSvVfFxwbmpeuMmcxclmSOkcs7wSyLHuyroh5xxgIFEpAAbWG',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 31.2681027, 'lng': 32.316558},
    'viewport': {'northeast': {'lat': 31.26936052989272,
      'lng': 32.31806997989272},
     'southwest': {'lat': 31.26666087010728, 'lng': 32.31537032010728}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pi

In [15]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address,Hotel Rating
25,Coalinga,36.1397,-120.3602,79.34,64,0,2.93,US,1635557418,Best Western Plus Coalinga Inn & Suites,"1786 Jayne Ave, Coalinga",4.5
34,San Patricio,28.0170,-97.5169,72.84,27,0,7.76,US,1635557309,Americas Best Value Inn Sinton,"8108 US-77, Sinton",3.4
49,Salalah,17.0151,54.0924,77.09,61,0,5.75,OM,1635557434,Salalah Gardens Hotel,Al Rabat Street، Salalah,4.8
59,Puerto Peñasco,31.3167,-113.5333,74.91,73,0,6.11,MX,1635557438,Las Palomas Beach & Golf Resort,"Blvd. Dr. Ernesto Guevara del Campo No.150, Pl...",4.6
60,Santa Cruz de Tenerife,28.4682,-16.2546,71.69,79,0,1.01,ES,1635557439,Iberostar Heritage Grand Mencey,"Calle Dr. Jose Naveiras, 38, Santa Cruz de Ten...",4.6


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content = hotel_info)


# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))